In [4]:
%load_ext autoreload
%autoreload 2
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
sys.path.append('../../src/optimal_explorer')
from llm_utils import llm_call
from pprint import pprint as pp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


So let us do 50 games for each with frontier LLMs - one with reasoning and one without.

# 2. Setup LLM Call

In [2]:
import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from sglang.test.test_utils import is_in_ci
from sglang.utils import wait_for_server, print_highlight, terminate_process

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

# This is equivalent to running the following command in your terminal

# python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b --host 0.0.0.0
local_model_name = "Qwen/Qwen3-4B" # "qwen/qwen3-4b" # "meta-llama/Llama-3.3-70B-Instruct" # "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"-- assert issue #  "Qwen/Qwen2.5-3B-Instruct" # "Qwen/QwQ-32B"
#
tp_size = (len(os.environ["CUDA_VISIBLE_DEVICES"]) + 1) // 2
server_process, port = launch_server_cmd(
    f"""
python3 -m sglang.launch_server --model-path {local_model_name} --host 0.0.0.0 --tp {tp_size}
"""# --tp {tp_size} mem_fraction_static=0.4 
)
wait_for_server(f"http://localhost:{port}")
print(port)

[2025-08-13 18:35:48] server_args=ServerArgs(model_path='Qwen/Qwen3-4B', tokenizer_path='Qwen/Qwen3-4B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen3-4B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, host='0.0.0.0', port=37044, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=241062498, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', log_level_http=None, log_requests

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(local_model_name)

messages = [
            {"role": "system", "content": 'You are a useless assistant that gives humorous answers.'},
            {"role": "user", "content": 'Why is the sky blue?'},
            {"role": "assistant", "content": 'It is because of the great king of Pokemons.'},
            {"role": "user", "content": 'How so?'},
            {"role": "assistant", "content": "<think>I think I should answer: just because.</think>"}
        ],

prompt_chat_str = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)[0][:-13]
prompt_chat_str
# tokens = tokenizer(prompt_chat_str, return_tensors="pt", add_special_tokens=False)
# tokenizer.decode(tokens['input_ids'][0])

'<|im_start|>system\nYou are a useless assistant that gives humorous answers.<|im_end|>\n<|im_start|>user\nWhy is the sky blue?<|im_end|>\n<|im_start|>assistant\nIt is because of the great king of Pokemons.<|im_end|>\n<|im_start|>user\nHow so?<|im_end|>\n<|im_start|>assistant\n<think>\nI think I should answer: just because.\n</think>'

In [12]:
import requests

url = f"http://localhost:{port}/generate"

response = requests.post(
    url,
    json={
        "text": prompt_chat_str,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 320,
        },
    },
)
print_highlight(response.json())

KeyboardInterrupt: 

In [10]:
print(port)

In [ ]:
from llm_utils import llm_call
out = await llm_call(
    model=local_model_name,
    temperature=1,
    url = f"http://localhost:{port}/v1/chat/completions",
    get_everything=True,
    reasoning_effort='high',
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': 'This is the game information:\nYou are playing a game of Wordle.\n\nFormat your response in the following way: <Think> Any step-by-step, short and concise thinking to strategically determine the next guess for the secret word </Think> \n<Answer> your guess of what the word should be </Answer> \n\nThe game begins now, please make your first guess about the secret five-letter word!\nYou are currently taking your attempt 2, and you have a total of 6 attempts.\nLook at the current belief state and give an answer based on it.Give an answer that leads to optimal exploration and do not be greedy unless it is the last attempt. Try to maximize the amount of information you have so that you can solve the task correctly.Belief: This is the start of the game. No beliefs right now.\nPlease format your response as: <Think> Any step-by-step, short and concise thinking to determine what the next guess should be </Think>\n <Answer> your answer in the correct format mentioned </Answer>'}]
)

[2025-08-13 18:42:31] Prefill batch. #new-seq: 1, #new-token: 142, #cached-token: 38, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-08-13 18:42:32] Decode batch. #running-req: 1, #token: 216, token usage: 0.00, cuda graph: True, gen throughput (token/s): 1.58, #queue-req: 0
[2025-08-13 18:42:32] Decode batch. #running-req: 1, #token: 256, token usage: 0.00, cuda graph: True, gen throughput (token/s): 71.61, #queue-req: 0
[2025-08-13 18:42:33] INFO:     127.0.0.1:51842 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-13 18:42:33] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 279, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-08-13 18:42:33] INFO:     127.0.0.1:51842 - "POST /generate HTTP/1.1" 200 OK


In [10]:
pp(out['choices'][0]['message']['content'], width=130)

('<think>\n'
 "Okay, the user is playing Wordle and needs my first guess. Since it's the start of the game and there are no previous beliefs, "
 'I should choose a word that\'s common and has a good balance of vowels and consonants. Words like "crane" or "fruit" are often '
 'good because they have multiple vowels which can help in figuring out the secret word. But maybe "crane" is too common? '
 'Alternatively, "apple" is a common first guess. Wait\n'
 'Considering the limited time by the user, I have to give the solution based on the thinking directly now.\n'
 '</think>.\n'
 '\n'
 ' <Answer>crane</Answer>')


In [19]:
# set PORT in run_baselines_paprika_frontier.py to the port number here:
print(port)

37044


In [53]:
from run_baselines_paprika_frontier import run_multiple_iterations_multiple_games

await run_multiple_iterations_multiple_games(
    num_games=40,
    list_envs=['wordle','mastermind'],
    models=[local_model_name],
    word_limits=[None],
    logs_file='./logs/paprika_qwen3_4B.jsonl',
    infos=['belief','history','both'],
)

Start running.
[2025-08-13 19:40:59] Prefill batch. #new-seq: 1, #new-token: 91, #cached-token: 77, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-08-13 19:40:59] Prefill batch. #new-seq: 5, #new-token: 1274, #cached-token: 275, token usage: 0.00, #running-req: 1, #queue-req: 0
[2025-08-13 19:40:59] Prefill batch. #new-seq: 30, #new-token: 8192, #cached-token: 2024, token usage: 0.01, #running-req: 6, #queue-req: 21
[2025-08-13 19:41:00] Prefill batch. #new-seq: 28, #new-token: 510, #cached-token: 8615, token usage: 0.04, #running-req: 35, #queue-req: 0
[2025-08-13 19:41:00] Prefill batch. #new-seq: 37, #new-token: 37, #cached-token: 12258, token usage: 0.01, #running-req: 63, #queue-req: 0
[2025-08-13 19:41:01] Decode batch. #running-req: 100, #token: 3188, token usage: 0.01, cuda graph: True, gen throughput (token/s): 1.17, #queue-req: 0
[2025-08-13 19:41:02] Decode batch. #running-req: 100, #token: 7188, token usage: 0.03, cuda graph: True, gen throughput (token/s): 4430.22

In [128]:
# don't forget to terminate the server when you're done!
terminate_process(server_process)

# 1. Load the Paprika Environments

In [ ]:
import pandas as pd
sys.path.append('../../paprika/') # Remove this when verl and paprika are installed in the same env
from llm_exploration.paprika_config_helper import PaprikaConfigHelper
from verl.interactions.paprika_interaction import PaprikaInteraction
from pprint import pprint as pp
paprika_games = ['twenty_questions', 'guess_my_city', 'murder_mystery', 'customer_service', 'wordle', 'cellular_automata', \
    'mastermind'] # 'battleship', 'minesweeper', 'bandit_bai_fixed_budget' 
import dotenv
dotenv.load_dotenv('../../.env')

Could not import JerichoInferenceEngine, so cannot use it!
Could not import VLLMInferenceEngine, so cannot use it!


True

In [ ]:
for env_name in paprika_games:
    config = PaprikaConfigHelper.create_config(env_name)
    config['belief_config']['style'] = 'none'

    interaction = PaprikaInteraction(config={})

    import builtins
    _original_print = builtins.print
    builtins.print = lambda *a, **k: None
    try:
        instance_id = await interaction.start_interaction(
            instance_id=None,
            scenario_id=None, # start a random scenario
            **config,
        )
    finally:
        builtins.print = _original_print
    num_scenarios = interaction.game_scenarios.__len__()
    print(f'{env_name} => {num_scenarios}')

twenty_questions => 367
guess_my_city => 185
murder_mystery => 50
customer_service => 200
wordle => 800
cellular_automata => 500
mastermind => 500


# 3. Belief Update

In [9]:
async def update_belief(
        curr_belief: str,
        action: str,
        response: str,
        model_name: str,
    ):


    user_content = f'''\
Look at the current belief and the agent's action and environment response on that belief.\
Compress the context, remove redundant information, and maintain important information about the game state \
needed to take optimal future actions.\
Current belief: {curr_belief}
Agent's action: {action}
Environment's response: {response}
Output the updated belief state inside <BELIEF> and </BELIEF> tags.\
Understand that only the generated belief is fed to the agent, so be sure to include all necessary information about game mechanics.'''

    messages = [
        {"role": "system", "content": 'You are a helpful assistant.'},
        {"role": "user", "content": user_content},
    ]

    if 'qwen' in model_name.lower():
        url = f"http://localhost:{port}/v1/chat/completions"
    else:
        url = None

    out = await llm_call(
        model=model_name,
        get_everything=True,
        reasoning_effort='high',
        messages=messages,
        url=url
    )

    import re
    content = out['choices'][0]['message']['content']
    match = re.search(r"<BELIEF>(.*?)</BELIEF>", content, re.DOTALL | re.IGNORECASE)
    if match:
        belief = match.group(1).strip()
    else:
        # fallback: return the whole content if tags not found
        belief = content.strip()
    
    if 'reasoning_details' in out['choices'][0]['message']:
        reasoning = out['choices'][0]['message']['reasoning_details'][0]['text']
    else:
        reasoning = None

    return belief, reasoning

In [10]:
async def take_action(
        belief: str,
        model_name: str,
    ):


    user_content = f'''\
Look at the current belief take the next action based on the belief.\
Take an action that leads to optimal exploration.\
Belief: {belief}
Output the action inside <ACTION> and </ACTION> tags.'''

    messages = [
        {"role": "system", "content": 'You are a helpful assistant.'},
        {"role": "user", "content": user_content},
    ]

    if 'qwen' in model_name.lower():
        url = f"http://localhost:{port}/v1/chat/completions"
    else:
        url = None

    out = await llm_call(
        model=model_name,
        url=url,
        get_everything=True,
        reasoning_effort='high',
        messages=messages
    )

    import re
    content = out['choices'][0]['message']['content']
    match = re.search(r"<\s*action\s*>(.*?)<\s*/\s*action\s*>", content, re.DOTALL | re.IGNORECASE)
    if match:
        action = match.group(1).strip()
    else:
        # fallback: return the whole content if tags not found
        action = content.strip()
    
    if 'reasoning_details' in out['choices'][0]['message']:
        reasoning = out['choices'][0]['message']['reasoning_details'][0]['text']
    else:
        reasoning = None

    return action, reasoning

# 4. Paprika Rollout with Belief

In [11]:
import asyncio

async def run_one_iteration_with_belief_llm(
        env_name: str,
        model_name: str,
        game_id: int,
    ):
    config = PaprikaConfigHelper.create_config(env_name)
    config['belief_config']['style'] = 'none'
    interaction = PaprikaInteraction(config={})

    import builtins
    _original_print = builtins.print
    builtins.print = lambda *a, **k: None
    try:
        instance_id = await interaction.start_interaction(instance_id=None, scenario_id=None, **config)
    finally:
        builtins.print = _original_print

    first_user_message = interaction.agent_conv.messages[0][1]
    attempts = 0
    game_history = []
    belief = f'This is the start of the game. The only available information right now are the game rules:\n{first_user_message}'
    max_attempts = interaction._instance_dict[instance_id]['max_turns']

    while attempts < max_attempts:
        
        attempts += 1

        action, action_reasoning = await take_action(belief, model_name)

        message = [
            {"role": "user", "content": f"Output the next action."},
            {"role": "assistant", "content": f"<action>{action}</action>"}
        ]
        done, response, score, additional_data = await interaction.generate_response(instance_id=instance_id, messages=message)
        
        belief, belief_reasoning = await update_belief(belief, action, response, model_name)

        game_history.append({
            "model": model_name,
            "game_id": str(game_id),
            "env": env_name,
            "attempt": attempts,
            "guess": action,
            "response": response,
            "score": score,
            "done": done,
            "data": additional_data,
            "belief": belief,
            "action_reasoning": action_reasoning,
            "belief_reasoning": belief_reasoning,
        })

        if "Goal reached" in response:
            break
    
    print(f'.', end='', flush=True)
    
    return game_history

async def run_multiple_iterations_multiple_games(
        num_games: int,
        list_envs,
        models,
        logs_file='./logs/paprika_local.jsonl',
    ):
    import json

    tasks = []
    for model in models:
        for env_name in list_envs:
            for game_id in range(num_games):
                tasks.append(run_one_iteration_with_belief_llm(env_name, model, game_id))

    results = await asyncio.gather(*tasks)

    # Flatten results and write to file
    with open(logs_file, "a") as f:
        for game_history in results:
            for entry in game_history:
                f.write(json.dumps(entry) + "\n")

# 4. Metrics

In [55]:
import pandas as pd
from pprint import pprint as pp
import json

logs_file = './logs/paprika_qwen3_4B.jsonl'
with open(logs_file, 'r') as f:
    data = [json.loads(line) for line in f]
df = pd.DataFrame(data)

logs_file = './logs/paprika_frontier_v6.jsonl'
with open(logs_file, 'r') as f:
    data = [json.loads(line) for line in f]

df2 = pd.DataFrame(data)
df = pd.concat([df, df2])

In [22]:
print(df.columns)

Index(['model', 'game_id', 'env', 'attempt', 'info', 'raw_guess', 'guess',
       'response', 'word_limit', 'score', 'done', 'data', 'belief',
       'action_reasoning', 'belief_reasoning'],
      dtype='object')


In [47]:
df.head(3)

,model,game_id,env,attempt,info,raw_guess,guess,response,word_limit,score,done,data,belief,action_reasoning,belief_reasoning,model_info
0,Qwen/Qwen3-4B (belief),5,wordle,1,belief,"<think>\nOkay, the user is playing Wordle and ...",stare,"First letter, s, is not in the target word \nS...",None,1,True,"{'agent_game_scenario': 'word', 'env_game_scen...","<think>\nOkay, let's see. The user is playing ...",,,Qwen/Qwen3-4B (belief)
1,Qwen/Qwen3-4B (belief),5,wordle,2,belief,"<think>\nOkay, the user is playing Wordle and ...",trend,"First letter, t, is correct and in the correct...",None,2,True,"{'agent_game_scenario': 'word', 'env_game_scen...","<think>\nOkay, let's see. The user is playing ...",,,Qwen/Qwen3-4B (belief)
2,Qwen/Qwen3-4B (belief),5,wordle,3,belief,"<think>\nOkay, the user is playing Wordle and ...",tried,Goal reached,None,3,True,"{'agent_game_scenario': 'word', 'env_game_scen...","The secret word is confirmed as ""tried"". The l...",,,Qwen/Qwen3-4B (belief)


In [56]:
df[df['model'].str.contains('qwen', case=False)].head(10)[['attempt', 'guess']]

,attempt,guess
0,1,crane
1,2,STRAW
2,1,stone
3,2,beach
4,3,"<think>\nOkay, let's try to figure out the sec..."
5,4,BAKER
6,1,...
7,2,stare
8,3,STAIR
9,1,crane


In [31]:
df[df['model'].str.contains('qwen', case=False)].head(10)['raw_guess'].values

array(['<think>\nOkay, the user is playing Wordle and needs help with their first guess. Let me think. Since it\'s the start of the game, there are no previous beliefs. I need to choose a word that\'s common and has a good chance of being the secret word. Wordle words are usually five letters, so I should pick a common one. Words like "crane", "apple", or "stone" are often used. But maybe "crane" is too specific. "Apple" is a good choice because it\'s common and has a lot of vowels. Alternatively, "stare" is another option. Wait, the user is on their second attempt, but the belief says it\'s the start. Maybe the user is confused. Wait, the initial message says "You are currently taking your attempt 2, and you have a total of 6 attempts." But the belief is that it\'s the start. That seems conflicting. Maybe the user is in a situation where they have already made one guess and are on the second attempt. But the belief says "This is the start of the game. No beliefs right now." So perhaps

In [57]:
print([len(x) for x in df[df['model'].str.contains('qwen', case=False)].head(10)['belief'].values])

[151, 145, 0, 0, 0, 0, 0, 0, 0, 0]


In [58]:
if not df.iloc[0]['word_limit']:
    df['word_limit'] = 'None'

In [59]:
df['info'].value_counts()

info
belief     9462
both       9194
history    8863
Name: count, dtype: int64

In [60]:
df['model_info'] = df['model'].astype(str) + ' (' + df['info'].astype(str) + ')'

In [61]:
df['model'] = df['model_info']

In [62]:
won = df.groupby(['model', 'game_id', 'env', 'word_limit'])['response'].apply(
        lambda responses: any('goal reached' in resp.lower() for resp in responses))
won.agg(['mean', 'std', 'count']).reset_index()

,index,response
0,mean,0.585833
1,std,0.492680
2,count,2400.000000


In [63]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

def summarize_game_outcomes(df):
    df['response'] = df['response'].str.lower()
    grouped = df.groupby(['model', 'game_id', 'env', 'word_limit'])['response'].apply(
        lambda responses: any('goal reached' in resp for resp in responses)
    ).reset_index(name='won')
    return grouped

def plot_win_rates(summary_df):
    # Compute mean and std of win rates for each group
    stats = summary_df.groupby(['env', 'model', 'word_limit'])['won'].agg(['mean', 'std', 'count']).reset_index()
    stats['success_rate'] = stats['mean'] * 100
    # Standard error of the mean (SEM)
    stats['sem'] = stats['std'] / np.sqrt(stats['count'])
    stats['sem'] = stats['sem'].fillna(0)
    stats['success_rate_sem'] = stats['sem'] * 100

    envs = stats['env'].unique()
    models = stats['model'].unique()[[0, 2, 1, 3, 5, 4, 6, 8, 7, 9, 11, 10]]
    word_limits = sorted(stats['word_limit'].unique())

    colors = [
        "#ADD8E6",  # light blue
        "#4682B4",  # medium blue
        "#003366",  # dark blue
        "#FFB6C1",  # light red
        "#FF6347",  # medium red
        "#8B0000",  # dark red
        "#90EE90",  # light green
        "#32CD32",  # medium green
        "#006400",  # dark green
        "#E6E6FA",  # light purple
        "#9370DB",  # medium purple
        "#800080",  # dark purple
    ]
    color_map = {model: colors[i % len(colors)] for i, model in enumerate(models)}

    # Create subplots: rows = word_limit values, cols = envs
    fig = make_subplots(
        rows=len(word_limits), cols=len(envs),
        subplot_titles=[f"{env}" for env in envs],
        shared_yaxes=True,
        vertical_spacing=0.2 / len(word_limits),
        horizontal_spacing=0.03
    )

    for r, wl in enumerate(word_limits, start=1):
        for c, env in enumerate(envs, start=1):
            for model in models:
                subset = stats[
                    (stats['env'] == env) &
                    (stats['model'] == model) &
                    (stats['word_limit'] == wl)
                ]
                if not subset.empty:
                    # Plot bar with error bar (standard error of mean)
                    fig.add_trace(
                        go.Bar(
                            x=[model],
                            y=subset['success_rate'],
                            name=model,
                            marker_color=color_map[model],
                            width=0.8,
                            showlegend=(r == 1 and c == 1),
                            error_y=dict(
                                type='data',
                                array=subset['success_rate_sem'],
                                visible=True,
                                color='black',
                                thickness=1,
                                width=4,
                            ),
                            hovertemplate=(
                                f"Env: {env}<br>"
                                f"Word Limit: {wl}<br>"
                                f"Model: {model}<br>"
                                f"Success Rate: {{y:.2f}}%<br>"
                                f"SEM: {subset['success_rate_sem'].values[0]:.2f}%<br>"
                                f"N: {subset['count'].values[0]}"
                            ),
                        ),
                        row=r, col=c
                    )
            # # Add row label for word_limit
            # if c == 1:
            #     fig.add_annotation(
            #         text=f"Word Limit: {wl}",
            #         xref="paper",
            #         yref="paper",
            #         x=0.1,
            #         y=0.95 - ((r - 1) / len(word_limits)),
            #         showarrow=False,
            #         font=dict(size=14)
            #     )

    # Update y-axis
    for r in range(1, len(word_limits) + 1):
        for c in range(1, len(envs) + 1):
            fig.update_yaxes(
                range=[0, 100],
                showgrid=True,
                gridcolor='lightgray',
                row=r, col=c
            )

    # Update x-axis to remove tick labels
    for r in range(1, len(word_limits) + 1):
        for c in range(1, len(envs) + 1):
            fig.update_xaxes(
                showticklabels=False,
                row=r, col=c
            )

    # Fixed subplot size
    fig_width = 180 * len(envs)
    fig_height = 200 * len(word_limits)

    # Layout with horizontal legend
    fig.update_layout(
        height=fig_height + 150,
        width=fig_width,
        template='simple_white',
        font=dict(family='Computer Modern, serif', size=16),
        barmode='group',
        showlegend=True,
        legend=dict(
            title='Models',
            orientation='h',
            yanchor='bottom',
            y=1.18,
            xanchor='center',
            x=0.5,
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='black',
            borderwidth=1
        ),
        margin=dict(t=100, b=50, l=80, r=20),
        plot_bgcolor='white'
    )

    fig.update_yaxes(title_text="Avg. Success Rate", row=1, col=1)

    return fig

In [64]:
df_belief = df.loc[df['info'] == 'belief'].copy()
df_history = df.loc[df['info'] == 'history'].copy()
summary_df = summarize_game_outcomes(df)
summary_df_belief = summarize_game_outcomes(df_belief)
summary_df_history = summarize_game_outcomes(df_history)

In [66]:
fig = plot_win_rates(summary_df)
fig.update_layout(
        
        legend=dict(
            title='Models',
            orientation='h',
            yanchor='bottom',
            y=1.18,
            xanchor='center',
            x=0.5,
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='black',
            borderwidth=1
        ),
        margin=dict(t=100, b=50, l=80, r=20),
        plot_bgcolor='white'
    )
fig.show()

In [113]:
stats = summary_df.groupby(['env', 'model', 'word_limit'])['won'].agg(['mean', 'std', 'count']).reset_index()
stats

,env,model,word_limit,mean,std,count
0,customer_service,deepseek/deepseek-chat (belief),None,0.475,0.505736,40
1,customer_service,deepseek/deepseek-chat (both),None,0.700,0.464095,40
2,customer_service,deepseek/deepseek-chat (history),None,0.775,0.422902,40
3,customer_service,deepseek/deepseek-r1 (belief),None,0.725,0.452203,40
4,customer_service,deepseek/deepseek-r1 (both),None,0.650,0.483046,40
5,customer_service,deepseek/deepseek-r1 (history),None,0.500,0.506370,40
6,customer_service,google/gemini-2.5-pro (belief),None,0.900,0.303822,40
7,customer_service,google/gemini-2.5-pro (both),None,0.950,0.220721,40
8,customer_service,google/gemini-2.5-pro (history),None,1.000,0.000000,40
9,guess_my_city,deepseek/deepseek-chat (belief),None,0.750,0.438529,40


In [56]:
fig = plot_win_rates(summary_df_history)
fig.show()

In [39]:
df.columns

Index(['model', 'game_id', 'env', 'attempt', 'raw_guess', 'guess', 'response',
       'word_limit', 'score', 'done', 'data', 'belief', 'action_reasoning',
       'belief_reasoning'],
      dtype='object')

In [78]:
wordle_sample = df[
    (df['model'] == 'deepseek/deepseek-r1 (belief)') & \
        (df['env'] == 'twenty_questions') & \
            (df['game_id'] == str(0)) & \
            (df['info'] == 'belief')
            ]
print(len(wordle_sample))

20


In [79]:
wordle_sample

,model,game_id,env,attempt,info,raw_guess,guess,response,word_limit,score,done,data,belief,action_reasoning,belief_reasoning,model_info
554,deepseek/deepseek-r1 (belief),0,twenty_questions,1,belief,<Think> Start by distinguishing between common...,Is the vegetable typically grown underground?,yes.,None,1.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...",The vegetable is a root vegetable typically gr...,"Okay, so I'm trying to guess a vegetable in 20...","Okay, so I need to update the belief state aft...",deepseek/deepseek-r1 (belief)
555,deepseek/deepseek-r1 (belief),0,twenty_questions,2,belief,<Think> Asking whether the vegetable is a bulb...,"Is the vegetable a bulb, such as onion or garlic?",no.,None,2.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...",The vegetable is a root vegetable typically gr...,"Okay, so the user is playing 20 Questions and ...","Okay, let's see. The user is playing 20 Questi...",deepseek/deepseek-r1 (belief)
556,deepseek/deepseek-r1 (belief),0,twenty_questions,3,belief,<Think> Asking whether the vegetable is a tube...,"Is the vegetable classified as a tuber (e.g., ...",yes.,None,3.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...",The vegetable is confirmed to be a tuber (e.g....,"Okay, let's see. The belief state says it's a ...","Okay, let's break this down. The user provided...",deepseek/deepseek-r1 (belief)
557,deepseek/deepseek-r1 (belief),0,twenty_questions,4,belief,<Think>Since the belief state confirms the veg...,Is the vegetable a potato?,no.,None,4.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...","The vegetable is confirmed to be a tuber, but ...","Okay, so the belief state says the vegetable i...","Okay, let's break down how to update the belie...",deepseek/deepseek-r1 (belief)
558,deepseek/deepseek-r1 (belief),0,twenty_questions,5,belief,<Think>\nTo differentiate between the remainin...,Is the vegetable typically smaller than a pota...,yes.,None,5.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...","The vegetable is a tuber, confirmed not to be ...","Okay, let's see. The user is trying to figure ...","Okay, let me process this step by step. The cu...",deepseek/deepseek-r1 (belief)
559,deepseek/deepseek-r1 (belief),0,twenty_questions,6,belief,<Think> The belief strongly points to Jerusale...,"Does the vegetable have thin, pale brown skin?",yes.,None,6.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...",The vegetable is a tuber smaller than a potato...,"Okay, let's see. The belief state says the veg...","Okay, let me see. The user is playing 20 Quest...",deepseek/deepseek-r1 (belief)
560,deepseek/deepseek-r1 (belief),0,twenty_questions,7,belief,<Think> The vegetable is likely a sunchoke bas...,Is the vegetable known for having a slightly n...,yes.,None,7.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...",The vegetable is confirmed to have a slightly ...,"Okay, so based on the belief state, the curren...","Okay, the user is playing 20 Questions trying ...",deepseek/deepseek-r1 (belief)
561,deepseek/deepseek-r1 (belief),0,twenty_questions,8,belief,<Think>The belief state strongly indicates Jer...,Is the vegetable a Jerusalem artichoke (suncho...,no.,None,8.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...","The vegetable is a small, knobby tuber with th...","Okay, so based on the belief state, the assist...","Okay, so the user was pretty confident that th...",deepseek/deepseek-r1 (belief)
562,deepseek/deepseek-r1 (belief),0,twenty_questions,9,belief,<Think>To distinguish between Yacon (eaten raw...,Is the vegetable commonly eaten raw?,no.,None,9.0,True,"{'agent_game_scenario': 'vegetable', 'env_game...","The vegetable is a small, knobby tuber with th...","Okay, so I'm trying to figure out which vegeta...","Okay, let me try to update the belief state ba...",deepseek/deepseek-r1 (belief)
563,deepseek/deepseek-r1 (belief),0,twenty_questions,10,belief,<Think> To distinguish between Crosne and Chin...,Is the vegetable typically know

In [85]:
pp(wordle_sample.iloc[19]['belief'])

('The vegetable is **not** Burdock root, Crosne, potato, carrot, beet, Chinese '
 'yam, water chestnut, jicama, Jerusalem artichoke, lotus root, water caltrop, '
 'galangal, **gourd (including ivy gourd)**, Hosui pear-shaped yam, **or '
 'winged yam (Dioscorea alata)**.  \n'
 '**Confirmed traits**:  \n'
 '- Crunchy when cooked.  \n'
 '- Small, knobby, non-round/non-smooth appearance.  \n'
 '- Pale brown skin with nutty/sweet flavor.  \n'
 '- Asian origin (common in Asian cuisine).  \n'
 '- Primarily used as a culinary vegetable (**not** a spice/herb).  \n'
 '- Commonly stir-fried in Asian dishes.  \n'
 '**Narrowed possibilities**:  \n'
 '- Excludes winged/lobed shapes. Focus on knobby, irregularly shaped '
 'tubers/roots.  \n'
 '- Candidates: Lesser-known Southeast Asian tubers (e.g., *greater yam '
 'variants*, *Taro cultivars* with knobby texture) or fibrous-textured roots '
 'used in Thai/Malay/Indonesian stir-fries.  \n'
 '- Key questions remaining: Distinguish between starchy vs. 